In [ ]:
# numpy experimentation
a=np.ones((10))
b=np.zeros(10)
b[1]=1
print a.shape, " ", b.shape
c =1.2*np.outer(a,b)
print (a*b)
print np.square(c)
print c
print c[:,1].shape
a[0][b==0]=4
print a

In [1]:
import numpy as np
import struct
import sys
import gzip

def read_file(filename):
    with gzip.open(filename,'rb') as fp:
        zero, data_type, dims = struct.unpack('>HBB', fp.read(4))
        shape = tuple(struct.unpack('>I', fp.read(4))[0] for d in range(dims))
        np_array = np.frombuffer(fp.read(), dtype=np.uint8).reshape(shape)
    return np_array

def preprocess_type1(image_file, label_file):
    images = read_file(image_file)
    labels = read_file(label_file)
    if (len(labels) > 10000):
        labels = labels[:10000]
        images = images[:10000]    
    images = images/255.0
    images = images.reshape( (10000, 784))

    labels = labels.reshape(-1,1)
    data = np.concatenate((images, labels), axis=1)
    np.random.shuffle(data)
    return data

def preprocess_type2(image_file, label_file):
    images = read_file(image_file)
    labels = read_file(label_file)
    if (len(labels) > 10000):
        labels = labels[:10000]
        images = images[:10000]    
    images = images/255.0
    images_sample = np.zeros((10000, 14, 14))
    for i,image in enumerate(images):
        for j in range(14):
            for k in range(14):
                images_sample[i][j][k] = max(image[j*2][k*2], image[j*2][k*2+1],
                                                 image[j*2+1][k*2], image[j*2+1][k*2+1])
    images_sample = images_sample.reshape( (10000, 196))

    labels = labels.reshape(-1,1)
    data = np.concatenate((images_sample, labels), axis=1)
    np.random.shuffle(data)
    return data

def get_features_labels(data, bias):
    examples = data[:,:-1]
    labels = data[:,-1]
    classifier_labels = np.zeros((10, len(labels)))
    for i in range(10):
        classifier_labels[i][labels == i] = 1
    examples = np.append(examples, bias, 1)
    return examples, labels, classifier_labels

In [2]:
def inference(test_data, weights):
    data_size = len(test_data)
    bias = np.ones((data_size,1))
    examples, labels, classifier_labels = get_features_labels(test_data, bias)
    prediction = np.ones(data_size, dtype = int)
    correct = 0
    for i, example in enumerate(examples):
        activation_values = sigmoid(np.sum(weights*example, axis = 1))
        prediction[i] = np.argmax(activation_values)
        if prediction[i] == labels[i]:
            correct += 1
    accuracy = correct*1.0/data_size
    return prediction, labels, accuracy

In [3]:
from numpy import linalg as LA
def sigmoid(value):
    value[value > 100] = 100
    value[value < -100] = -100
    return 1/(1+np.exp(-value))

In [4]:
path = "../hw2/DATA_FOLDER/"
train_data_path = path + '/train-images-idx3-ubyte.gz'
train_labels_path = path + '/train-labels-idx1-ubyte.gz'
test_data_path = path + '/t10k-images-idx3-ubyte.gz'
test_labels_path = path + '/t10k-labels-idx1-ubyte.gz'
train_data_type1 = preprocess_type1(path + '/train-images-idx3-ubyte.gz', path + '/train-labels-idx1-ubyte.gz')
# print train_data[1]
test_data_type1 = preprocess_type1(path + '/t10k-images-idx3-ubyte.gz', path + '/t10k-labels-idx1-ubyte.gz')

train_data_type2 = preprocess_type2(path + '/train-images-idx3-ubyte.gz', path + '/train-labels-idx1-ubyte.gz')
# print train_data[1]
test_data_type2 = preprocess_type2(path + '/t10k-images-idx3-ubyte.gz', path + '/t10k-labels-idx1-ubyte.gz')



In [9]:
def train_gd(train_data, num_epoches, learning_rate, lamda, test_data):
    data_size = len(train_data)
    feature_length = train_data.shape[1]
    weights = np.random.uniform(-0.1,0.1,[10,feature_length])
    bias = np.ones((data_size,1))
    prev_loss = 1000
    for epoch in range(num_epoches):
        loss = np.zeros(10)
        np.random.shuffle(train_data)
        examples, labels, classifier_labels = get_features_labels(train_data, bias)
        delta_weights = np.zeros((10,feature_length))
        for i,example in enumerate(examples):
            z = np.sum(weights*example, axis = 1)
            y_pred = sigmoid(z)
            delta_weights += learning_rate*np.outer(y_pred-classifier_labels[:,i],example)
            loss+=classifier_labels[:,i]*np.log(y_pred+0.0000000001)+(1-classifier_labels[:,i])*np.log(1-y_pred+0.0000000001)
        
        weights = weights - delta_weights - learning_rate*lamda*weights
        loss = loss*-1.0/data_size + lamda*np.sum(np.square(weights), axis = 1)/2.0
        loss = np.sum(loss)/10.0
        train_prediction, train_labels, train_accuracy = inference(train_data[:train_data_size], weights)
        test_prediction, test_labels, test_accuracy = inference(test_data, weights)
        print "epoch ", epoch, ": Training loss: ", loss, " Training Accuracy: ", train_accuracy, " Test Accuracy: ", test_accuracy
        delta_loss = prev_loss - loss
        prev_loss = loss
        if delta_loss > 0 and delta_loss < 0.0001 and epoch > 100:
            return weights, loss
    return weights, loss 

In [13]:
def train_sgd(train_data, num_epoches, learning_rate, lamda, test_data):
    data_size = len(train_data)
    feature_length = train_data.shape[1]
    weights = np.random.uniform(-0.1,0.1,[10,feature_length])
    bias = np.ones((data_size,1))
    prev_loss = 1000
    batch_size_loss = 1000
    for epoch in range(num_epoches):
        loss = np.zeros(10)
        np.random.shuffle(train_data)
        examples, labels, classifier_labels = get_features_labels(train_data, bias)
        for i,example in enumerate(examples):
            delta_weights = np.zeros((10,feature_length))
            z = np.sum(weights*example, axis = 1)
            y_pred = sigmoid(z)
            delta_weights += learning_rate*np.outer(y_pred-classifier_labels[:,i],example)
            weights = weights - delta_weights - learning_rate*lamda*weights
            loss+=classifier_labels[:,i]*np.log(y_pred+0.0000000001)+(1-classifier_labels[:,i])*np.log(1-y_pred+0.0000000001)

            if i%batch_size_loss==0:
                loss = loss*-1.0/batch_size_loss + lamda*np.sum(np.square(weights), axis = 1)/2.0
                loss = np.sum(loss)/10.0
                
                train_prediction, train_labels, train_accuracy = inference(train_data[:train_data_size], weights)
                test_prediction, test_labels, test_accuracy = inference(test_data, weights)
                print "epoch ", epoch, ": Training loss: ", loss, " Training Accuracy: ", train_accuracy, " Test Accuracy: ", test_accuracy
                delta_loss = prev_loss - loss
                prev_loss = loss
                loss = np.zeros(10)
    #             print "delta loss ", delta_loss
                if delta_loss < 0.00007 and train_accuracy > 0.92:
                    return weights, loss
    return weights, loss 

In [ ]:
#stochastic gradient descent
train_data_size = 10000
epochs = 20
learning_rate = 0.01
lamda = 0.000001
# lamda = 0
gd_weights, train_loss = train_sgd(train_data_type1[:10000], epochs, learning_rate, lamda, test_data_type1)
test_prediction, test_labels, test_accuracy = inference(test_data_type1, gd_weights)
print "test accuracy ", test_accuracy, " training loss: ", train_loss

In [ ]:
gd_weights, train_loss = train_sgd(train_data_type2[:10000], epochs, learning_rate, lamda, test_data_type2)
test_prediction, test_labels, test_accuracy = inference(test_data_type2, gd_weights)
print "test accuracy ", test_accuracy, " training loss: ", train_loss

In [14]:
#batch gradient descent

train_data_size = 10000
epochs = 300
learning_rate = 0.0001
lamda = 0.0001
# lamda = 0
gd_weights, train_loss = train_gd(train_data_type1[:10000], epochs, learning_rate, lamda, test_data_type1)
test_prediction, test_labels, test_accuracy = inference(test_data_type1, gd_weights)
print "test accuracy ", test_accuracy, " training loss: ", train_loss

epoch  0 : Training loss:  0.749490814214  Training Accuracy:  0.1001  Test Accuracy:  0.098
epoch  1 : Training loss:  1.35691121953  Training Accuracy:  0.108  Test Accuracy:  0.1034
epoch  2 : Training loss:  0.89492671952  Training Accuracy:  0.2397  Test Accuracy:  0.2363
epoch  3 : Training loss:  0.499278318433  Training Accuracy:  0.5862  Test Accuracy:  0.5779
epoch  4 : Training loss:  0.225907814483  Training Accuracy:  0.7549  Test Accuracy:  0.7394
epoch  5 : Training loss:  0.169404270122  Training Accuracy:  0.7589  Test Accuracy:  0.7518
epoch  6 : Training loss:  0.157214788406  Training Accuracy:  0.7384  Test Accuracy:  0.7328
epoch  7 : Training loss:  0.182833599461  Training Accuracy:  0.7941  Test Accuracy:  0.7835
epoch  8 : Training loss:  0.139557626532  Training Accuracy:  0.7977  Test Accuracy:  0.7882
epoch  9 : Training loss:  0.139282669497  Training Accuracy:  0.7694  Test Accuracy:  0.7562
epoch  10 : Training loss:  0.14154441417  Training Accuracy:  0

epoch  87 : Training loss:  0.0799596047451  Training Accuracy:  0.9018  Test Accuracy:  0.8959
epoch  88 : Training loss:  0.0798090631522  Training Accuracy:  0.902  Test Accuracy:  0.896
epoch  89 : Training loss:  0.0796607593116  Training Accuracy:  0.9024  Test Accuracy:  0.8962
epoch  90 : Training loss:  0.0795146295054  Training Accuracy:  0.9027  Test Accuracy:  0.8964
epoch  91 : Training loss:  0.0793706227519  Training Accuracy:  0.9028  Test Accuracy:  0.8966
epoch  92 : Training loss:  0.0792286813527  Training Accuracy:  0.9028  Test Accuracy:  0.8967
epoch  93 : Training loss:  0.0790887569769  Training Accuracy:  0.903  Test Accuracy:  0.8968
epoch  94 : Training loss:  0.0789507970044  Training Accuracy:  0.9034  Test Accuracy:  0.8968
epoch  95 : Training loss:  0.0788147557166  Training Accuracy:  0.9037  Test Accuracy:  0.8971
epoch  96 : Training loss:  0.0786805848266  Training Accuracy:  0.9037  Test Accuracy:  0.8972
epoch  97 : Training loss:  0.0785482411684

In [11]:
gd_weights, train_loss = train_gd(train_data_type2[:10000], epochs, learning_rate, lamda, test_data_type2)
test_prediction, test_labels, test_accuracy = inference(test_data_type2, gd_weights)
print "test accuracy ", test_accuracy, " training loss: ", train_loss

epoch  0 : Training loss:  0.725195477227  Training Accuracy:  0.1444  Test Accuracy:  0.1453
epoch  1 : Training loss:  0.731428871801  Training Accuracy:  0.2235  Test Accuracy:  0.2256
epoch  2 : Training loss:  0.505359535526  Training Accuracy:  0.3745  Test Accuracy:  0.3866
epoch  3 : Training loss:  0.329317922611  Training Accuracy:  0.6728  Test Accuracy:  0.6655
epoch  4 : Training loss:  0.249179726576  Training Accuracy:  0.7373  Test Accuracy:  0.7335
epoch  5 : Training loss:  0.219714681822  Training Accuracy:  0.7971  Test Accuracy:  0.7925
epoch  6 : Training loss:  0.207415367117  Training Accuracy:  0.8072  Test Accuracy:  0.8052
epoch  7 : Training loss:  0.200127428926  Training Accuracy:  0.8154  Test Accuracy:  0.8111
epoch  8 : Training loss:  0.194126886143  Training Accuracy:  0.8202  Test Accuracy:  0.8197
epoch  9 : Training loss:  0.189038139602  Training Accuracy:  0.8259  Test Accuracy:  0.8228
epoch  10 : Training loss:  0.184727270167  Training Accurac

epoch  87 : Training loss:  0.157601635198  Training Accuracy:  0.8803  Test Accuracy:  0.8783
epoch  88 : Training loss:  0.157819981689  Training Accuracy:  0.8806  Test Accuracy:  0.8786
epoch  89 : Training loss:  0.158039909792  Training Accuracy:  0.8808  Test Accuracy:  0.8785
epoch  90 : Training loss:  0.158261344763  Training Accuracy:  0.8808  Test Accuracy:  0.8787
epoch  91 : Training loss:  0.15848421523  Training Accuracy:  0.8813  Test Accuracy:  0.8788
epoch  92 : Training loss:  0.158708453014  Training Accuracy:  0.8816  Test Accuracy:  0.879
epoch  93 : Training loss:  0.158933992964  Training Accuracy:  0.8818  Test Accuracy:  0.8793
epoch  94 : Training loss:  0.159160772806  Training Accuracy:  0.882  Test Accuracy:  0.8794
epoch  95 : Training loss:  0.159388732991  Training Accuracy:  0.8822  Test Accuracy:  0.8795
epoch  96 : Training loss:  0.159617816566  Training Accuracy:  0.8822  Test Accuracy:  0.8796
epoch  97 : Training loss:  0.159847969038  Training 

epoch  173 : Training loss:  0.178328135505  Training Accuracy:  0.8906  Test Accuracy:  0.8888
epoch  174 : Training loss:  0.178570379657  Training Accuracy:  0.8905  Test Accuracy:  0.8888
epoch  175 : Training loss:  0.178812434904  Training Accuracy:  0.8905  Test Accuracy:  0.8888
epoch  176 : Training loss:  0.179054298836  Training Accuracy:  0.8906  Test Accuracy:  0.8888
epoch  177 : Training loss:  0.179295969137  Training Accuracy:  0.8905  Test Accuracy:  0.8888
epoch  178 : Training loss:  0.179537443592  Training Accuracy:  0.8905  Test Accuracy:  0.889
epoch  179 : Training loss:  0.179778720077  Training Accuracy:  0.8907  Test Accuracy:  0.889
epoch  180 : Training loss:  0.180019796562  Training Accuracy:  0.8908  Test Accuracy:  0.8891
epoch  181 : Training loss:  0.180260671099  Training Accuracy:  0.8907  Test Accuracy:  0.8892
epoch  182 : Training loss:  0.18050134183  Training Accuracy:  0.8906  Test Accuracy:  0.8892
epoch  183 : Training loss:  0.180741806976

epoch  259 : Training loss:  0.198358410652  Training Accuracy:  0.8973  Test Accuracy:  0.8936
epoch  260 : Training loss:  0.198581422734  Training Accuracy:  0.8973  Test Accuracy:  0.8937
epoch  261 : Training loss:  0.198804213338  Training Accuracy:  0.8974  Test Accuracy:  0.8938
epoch  262 : Training loss:  0.199026782931  Training Accuracy:  0.8975  Test Accuracy:  0.8937
epoch  263 : Training loss:  0.199249131983  Training Accuracy:  0.8974  Test Accuracy:  0.8938
epoch  264 : Training loss:  0.199471260968  Training Accuracy:  0.8975  Test Accuracy:  0.8938
epoch  265 : Training loss:  0.199693170366  Training Accuracy:  0.8975  Test Accuracy:  0.8938
epoch  266 : Training loss:  0.199914860661  Training Accuracy:  0.8977  Test Accuracy:  0.8939
epoch  267 : Training loss:  0.200136332342  Training Accuracy:  0.8978  Test Accuracy:  0.8939
epoch  268 : Training loss:  0.200357585901  Training Accuracy:  0.8977  Test Accuracy:  0.8939
epoch  269 : Training loss:  0.200578621

In [ ]:
a= np.zeros((10,1))
sigmoid(a)